**Import packages**



In [368]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
import bs4
import re


In [369]:
suburbs= ("Sea Point",'Green Point',"Gardens")
def setup_driver():
    """"start the chrome driver and go to privateproperty.co.za
    and click on the allow cookies pop_up"""
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(5)
    driver.get('https://www.privateproperty.co.za/to-rent')
    time.sleep(2)
    driver.find_element(By.XPATH,'//*[@id="consentManager"]/div/div/button[2]').click()
    return driver
driver = setup_driver()
def original_search(driver,suburb)->str:
    """get base url for that specific suburb search"""
    # find the search bar
    search_bar = driver.find_element(By.XPATH,'//*[@id="homePageAutoComplete"]/div/div/div/div[2]/form/div/input[1]')
    time.sleep(2)
    def send_keys_delay(controller,keys,delay=0.1):
        """"send keys one at a time"""
        for key in keys:
            controller.send_keys(key)
            time.sleep(delay)

    send_keys_delay(search_bar,suburb)
    time.sleep(3)
    first_search = driver.find_element(By.XPATH,'//*[@id="homePageAutoComplete"]/div/div/div/div[2]/form/div/div/div[1]').click()
    base_url = driver.current_url
    return base_url


base_url = original_search(driver, suburbs[0])
print(base_url)

<ipython-input-369-360cc8d2dfd5>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.177)
Stacktrace:
Backtrace:
	(No symbol) [0x010D37D3]
	(No symbol) [0x01068B81]
	(No symbol) [0x00F6B36D]
	(No symbol) [0x00F4E5DA]
	(No symbol) [0x00FBF0CB]
	(No symbol) [0x00FD0A56]
	(No symbol) [0x00FBB216]
	(No symbol) [0x00F90D97]
	(No symbol) [0x00F9253D]
	GetHandleVerifier [0x0134ABF2+2510930]
	GetHandleVerifier [0x01378EC1+2700065]
	GetHandleVerifier [0x0137C86C+2714828]
	GetHandleVerifier [0x01183480+645344]
	(No symbol) [0x01070FD2]
	(No symbol) [0x01076C68]
	(No symbol) [0x01076D4B]
	(No symbol) [0x01080D6B]
	BaseThreadInitThunk [0x756B7D69+25]
	RtlInitializeExceptionChain [0x7736BB9B+107]
	RtlClearBits [0x7736BB1F+191]


In [ ]:

def make_soup(driver):
    
    soup = BeautifulSoup(driver.page_source,"html.parser")
    body_soup = soup.body
    return body_soup



In [ ]:
body_soup = make_soup(driver)

In [ ]:
driver.close()

In [ ]:

def number_of_pages(body_soup:bs4.element.Tag) -> int:
    """"find the number of listed places in the search to figure out how many pages to crawl"""
    listingcount = body_soup.find('div', {'class':'listingCount'})
    
    string1 = listingcount.text
    
    split_counts = re.findall('\d+',string1)
    
    this_page = int(split_counts[1])
    total_listings = int(split_counts[-1])
    pages_to_search = -(-total_listings//this_page)
    return pages_to_search 
    

In [ ]:
number_of_pages(body_soup)


4

In [ ]:
base_url

'https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/437'

In [ ]:
# once we have the count we visit the other pages:
page_count = number_of_pages(body_soup)

def make_subsequent_urls(page_count,base_url):
    urls_list = []
    for i in range(2,page_count+1):
        # i will be our page numbers to shoot into the url. 
        next_url = base_url + f"?page={i}"
        urls_list.append(next_url) 
    return urls_list # if it doesn't produce any urls, we moce on. 


In [ ]:
other_urls = make_subsequent_urls(page_count,base_url)
other_urls

['https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/437?page=2',
 'https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/437?page=3',
 'https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/437?page=4']

In [ ]:
import pandas as pd
def extract_listing_info(soup_body):
  """ Extract info from all listings on one page of privateproperty.co.za
      
  """
  data = []
  

  

  # FIND ALL PRODUCT ITEMS
  info_of_listings = soup_body.find_all('div',{'class':"infoHolder"}) 
  image_info = soup_body.find_all('div',{'class':"imageHolder"})
  overall_content =soup_body.find_all('a',{'class':"listingResult row"})
  assert len(info_of_listings)== len(image_info)==len(overall_content)
  print("BATCH SIZE:", len(info_of_listings))
  

  # Extracting to a dictionary
  counter = 0 
  for i,j,k in zip(info_of_listings,image_info,overall_content): # use i for all the info and j for the image info
      d = {'title':'','priceDescription':'','priceAdditionalDescriptor':'','propertyType':'', 'suburb':'',
           'number_of_bedrooms':'','number_of_bathrooms':'','number_of_garages':'',
           'available_from':'',"url":"","is_new":""}
           
      
      try:
        d['title']                      = i.find("div",{'class':"title"}).text
        d['priceDescription']           =i.find("div",{'class':"priceDescription"}).text
        d['priceAdditionalDescriptor']  = i.find("div",{'class':"priceAdditionalDescriptor"}).text
        d['propertyType']               =   i.find("div",{'class':"propertyType"}).text  
        d['suburb']                     = i.find("div",{'class':"suburb"}).text
        if i.find("div",{'class':"icon bedroom"}):
          d['number_of_bedrooms']         = i.find("div",{'class':"icon bedroom"}).find_previous_sibling().text
        else:
          d['number_of_bedrooms']        = None
        if i.find("div",{'class':"icon bathroom"}):
          d['number_of_bathrooms']        = i.find("div",{'class':"icon bathroom"}).find_previous_sibling().text
        else: d['number_of_bathrooms'] = None
        if i.find("div",{'class':"icon garage"}):
          d['number_of_garages']          = i.find("div",{'class':"icon garage"}).find_previous_sibling().text
        else: d['number_of_garages'] = None
        
        if j.find("div",{'class':"statusFlag flagAvailableNow"}):
          d['available_from']   = "Now"
        elif j.find("div",{'class':"statusFlag flagAvailableFrom"}):
          d['available_from']   = j.find("div",{'class':"statusFlag flagAvailableFrom"}).text
        else:
          d['available_from'] = None
        
        if j.find("div",{'class':"statusFlag flagNew"}):
          d['is_new']           = j.find("div",{'class':"statusFlag flagNew"}).text
        else: 
          d['is_new'] = "Not New"

        print("made it here")
        
        d['url']              = "https://www.privateproperty.co.za"+ k['href']
         
        
      except Exception as e: 
        print(e)
      data.append(d) 
      counter +=1  
  return data 

In [ ]:
extract_listing_info(body_soup)

BATCH SIZE: 24
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here


[{'title': '2 Bed Apartment in Sea Point',
  'priceDescription': 'R 35 000',
  'priceAdditionalDescriptor': '',
  'propertyType': 'Apartment',
  'suburb': 'Sea Point',
  'number_of_bedrooms': '2',
  'number_of_bathrooms': '2',
  'number_of_garages': None,
  'available_from': 'AVAILABLE FROM: 01 MAR 2023',
  'url': 'https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3639654',
  'is_new': 'NEW'},
 {'title': '3 Bed Apartment in Sea Point',
  'priceDescription': 'R 39 000',
  'priceAdditionalDescriptor': '',
  'propertyType': 'Apartment',
  'suburb': 'Sea Point',
  'number_of_bedrooms': '3',
  'number_of_bathrooms': '1.5',
  'number_of_garages': None,
  'available_from': 'AVAILABLE FROM: 01 MAY 2023',
  'url': 'https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3742429',
  'is_new': 'NEW'},
 {'title': 'Studio apartment in Sea Point',
  'priceDescription': 'R 15 000',
  'priceAdditionalDescriptor': '',
 

In [ ]:
list_of_data = extract_listing_info(driver)

In [ ]:

list_of_data # epic, could add some extra safety nets for making sure that all matches up

# next step is to scroll the next page

In [ ]:
def main():
    #steps
    all_data = []
    # go to url and get throught the cookie
    driver  = setup_driver()
    # choose the suburb from a list 
    suburbs= ("Sea Point",'Green Point',"Gardens")
    for suburb in suburbs:
        base_url = original_search(driver,suburb)
        soup_body = make_soup(driver) 
        number_of_pages_to_search = number_of_pages(soup_body)
        other_urls_list = make_subsequent_urls(number_of_pages_to_search,base_url)

        #get info from one page 
        data = extract_listing_info(soup_body)
        all_data.extend(data)
        
        for url in other_urls_list:
            driver.get(url)
            next_soup_body  = make_soup(driver)
            data_next = extract_listing_info(next_soup_body)
            all_data.extend(data_next)
        # go bac to the search page 
        time.sleep(5)
        driver.get('https://www.privateproperty.co.za/to-rent')
    driver.close()
    df = pd.DataFrame(all_data,index = range(len(all_data)))
    
    return df
    

In [ ]:
all_data_df = main()

# send it to the DB



<ipython-input-2-360cc8d2dfd5>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


BATCH SIZE: 24
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
BATCH SIZE: 24
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
BATCH SIZE: 24
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
made it here
BATCH SIZE: 6
made 

In [ ]:
all_data_df.head(50)


,title,priceDescription,priceAdditionalDescriptor,propertyType,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,available_from,url,is_new
0,2 Bed Apartment in Sea Point,R 35 000,,Apartment,Sea Point,2,2,None,AVAILABLE FROM: 01 MAR 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
1,3 Bed Apartment in Sea Point,R 39 000,,Apartment,Sea Point,3,1.5,None,AVAILABLE FROM: 01 MAY 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
2,Studio apartment in Sea Point,R 15 000,,Studio,Sea Point,None,1,None,Now,https://www.privateproperty.co.za/to-rent/west...,NEW
3,1 Bed Apartment in Sea Point,R 20 500,,Apartment,Sea Point,1,1,None,Now,https://www.privateproperty.co.za/to-rent/west...,NEW
4,2 Bed Apartment in Sea Point,R 15 000,,Apartment,Sea Point,2,1,None,AVAILABLE FROM: 01 MAR 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
5,2 Bed Flat in Sea Point,R 18 000,DEP R 18K,Flat,Sea Point,2,1,1,AVAILABLE FROM: 01 MAY 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
6,3 Bed Penthouse in Sea Point,R 55 000,,Penthouse,Sea Point,3,2,None,AVAILABLE FROM: 01 APR 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
7,2 Bed Apartment in Sea Point,R 25 000,,Apartment,Sea Point,2,1,None,AVAILABLE FROM: 01 APR 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
8,2 Bed Apartment in Sea Point,R 12 500,,Apartment,Sea Point,2,1,None,AVAILABLE FROM: 01 APR 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
9,2 Bed Apartment in Sea Point,R 38 500,,Apartment,Sea Point,2,2,1,Now,https://www.privateproperty.co.za/to-rent/west...,NEW


,title,priceDescription,priceAdditionalDescriptor,propertyType,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,available_from,url,is_new
0,4 Bed Apartment in Sea Point,R 60 000,DEP R 120K,Apartment,Sea Point,4,4.5,None,AVAILABLE FROM: 01 MAY 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
1,2 Bed Apartment in Sea Point,R 16 000,,Apartment,Sea Point,2,1,1,AVAILABLE FROM: 01 APR 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
2,3 Bed Penthouse in Sea Point,R 72 500,,Penthouse,Sea Point,3,3,2,Now,https://www.privateproperty.co.za/to-rent/west...,NEW
3,3 Bed Apartment in Sea Point,R 37 000,,Apartment,Sea Point,3,4,None,AVAILABLE FROM: 01 MAR 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
4,Studio apartment in Sea Point,R 8 500,,Studio,Sea Point,0.5,1,None,Now,https://www.privateproperty.co.za/to-rent/west...,NEW
...,...,...,...,...,...,...,...,...,...,...,...
84,2 Bed Apartment in Gardens,R 14 500,,Apartment,Gardens,2,1,None,Now,https://www.privateproperty.co.za/to-rent/west...,NEW
85,1 Bed Apartment in Gardens,R 10 000,,Apartment,Gardens,1,1,None,AVAILABLE FROM: 01 MAR 2023,https://www.privateproperty.co.za/to-rent/west...,Not New
86,3 Bed Duplex in Gardens,R 23 000,DEP R 34.5K,Duplex,Gardens,3,2,1,AVAILABLE FROM: 07 MAR 2023,https://www.privateproperty.co.za/to-rent/west...,NEW
87,Studio apartment in Gardens,R 9 000,,Studio,Gardens,0.5,1,None,Now,https://www.privateproperty.co.za/to-rent/west...,Not New


None

In [ ]:
A_list = []
A_dict = {"name":"","number":""}
A_dict["name"]="Fred"
A_dict["number"] = 1
A_list.append(A_dict)
print(A_list)
A_dict["name"]="Frank"
A_dict["number"] = 2
A_list.append(A_dict)
print(A_list)
# Interesting

[{'name': 'Fred', 'number': 1}]
[{'name': 'Frank', 'number': 2}, {'name': 'Frank', 'number': 2}]


In [ ]:
list_my

[{'name': 'Frank', 'number': 2}, {'name': 'Frank', 'number': 2}]

## Data cleaning 


In [1]:
import pandas as pd
df = pd.read_parquet("D:\Real estate data engineering project\saved_files\Full_Raw_Scrape20230228")
df.sample(30)
pp_df = df[df['url'].str.contains("privateproperty")]
p24_df = df[df['url'].str.contains("property24")]
ht_df = df[df['url'].str.contains("sahometraders")]

In [5]:
#pd.set_option('display.max_colwidth', None)
# the urls are all good now 

title                                                                                     Apartment / Flat
priceDescription                                                                                  R 15 000
priceAdditionalDescriptor                                                                                 
suburb                                                                                           Sea Point
number_of_bedrooms                                                                                        
number_of_bathrooms                                                                                      1
number_of_garages                                                                                        1
size                                                                                                      
available_from                                                                               AVAILABLE NOW
url                          https://

In [119]:
import numpy as np
# we want our columns to be useful for analysis
# let's say the type of building can be 
# 1. apartment, house, cottage, penthouse,commercial property, Industrial property, 
#ht_df.title.unique() 


mask_apt = ht_testet['title'].str.contains("Apartment|apartment|flat|Flat")
mask_house = ht_testet['title'].str.contains("house|House")
mask_commercial = ht_testet['title'].str.contains("commercial|Commercial")
mask_industrial = ht_testet['title'].str.contains("industrial|Industrial")
print(len(ht_testet["title"]),sum([mask_apt.sum(),mask_house.sum(),mask_commercial.sum(),mask_industrial.sum()]))
ht_testet['building type'] = np.select([mask_apt,mask_house,mask_commercial,mask_industrial],["apartment",'house','commercial property','industrial property'])

691 691


In [7]:
import numpy as np
def house_type_categories(df):
    """Make categorical column for building type"""
    mask_apt = df['title'].str.contains("Apartment|apartment|flat|Flat")
    mask_house = df['title'].str.contains("house|House")
    mask_commercial = df['title'].str.contains("commercial|Commercial")
    mask_industrial = df['title'].str.contains("industrial|Industrial")
    
    df['building type'] = np.select([mask_apt,mask_house,mask_commercial,mask_industrial],["apartment",'house','commercial property','industrial property'])
    return df
ht_clean = house_type_categories(ht_df)
pp_clean = house_type_categories(pp_df)
p24_clean = house_type_categories(p24_df)
p24_clean.head()

<ipython-input-7-d3373bb22907>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building type'] = np.select([mask_apt,mask_house,mask_commercial,mask_industrial],["apartment",'house','commercial property','industrial property'])


,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url,building type
0,2 Bedroom Apartment in Sea Point,R 60 000,,Sea Point,2,3,1,290 m²,AVAILABLE:01 APR,https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021/P112312422,apartment
1,1 Bedroom Apartment in Sea Point,R 25 000,,Sea Point,1,1,1,46 m²,AVAILABLE:01 APR,https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021/112204793,apartment
2,4 Bedroom Apartment in Sea Point,R 60 000,,Sea Point,4,4.5,2,203 m²,AVAILABLE:01 MAY,https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021/112279922,apartment
3,0.5 Bedroom Apartment,R 9 500,,Sea Point,0.5,1,,62 m²,AVAILABLE:01 APR,https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021/112370674,apartment
4,2 Bedroom Apartment,R 50 000,,Sea Point,2,2,4,,AVAILABLE:01 MAR,https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021/111968895,apartment


In [275]:

full_index = list(ht_testet.index)
now_index = list(ht_testet.loc[ht_testet['available_from'].str.contains("NOW",na=False),"available_from"].index)
other_index = set(full_index) - set(now_index)
ht_testet["available_from"]= ht_testet["available_from"].str.strip(" AVAILABLE:")
ht_testet.loc[now_index,"available_from"] = date.today()
ht_testet.loc[other_index,"available_from"] = ht_testet.loc[other_index,"available_from"] + " 2023"
ht_testet.loc[other_index,"available_from"] = pd.to_datetime(ht_testet.loc[other_index,"available_from"], format = "%d %b %Y",errors='coerce').dt.date

# I need to get better with this datetime stuff this is great practice.

# need to deal with the ones that say NOW 


In [276]:
#p24_testet['available_from'].value_counts()
ht_testet.head(20)

,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url,building type
0,4 Bedroom Apartment / Flat,R 60 000,None,Sea Point,4,4.5,2,203 m²,2023-05-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,apartment
1,2 Bedroom Apartment / Flat,R 40 000,None,Sea Point,2,2,2,None,2023-03-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,apartment
2,4 Bedroom Apartment / Flat,R 60 000,None,Sea Point,4,4.5,2,203 m²,2023-05-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,apartment
3,Commercial Property,R 15 180,None,Sea Point,None,None,None,92 m²,2023-02-27,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,commercial property
4,2 Bedroom Apartment / Flat,R 25 000,None,Sea Point,2,2,1,None,2023-04-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,apartment
5,2 Bedroom Apartment / Flat,R 22 000,None,Sea Point,2,1,1,90 m²,2023-04-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,apartment
6,4 Bedroom Apartment / Flat,R 45 000,None,Sea Point,4,3,1,160 m²,2023-04-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,apartment
7,Commercial Property,R 10 500,None,Sea Point,None,None,None,57 m²,2023-02-27,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,commercial property
8,2 Bedroom Apartment / Flat,R 15 000,None,Sea Point,2,1,1,None,2023-05-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,apartment
9,2 Bedroom Apartment / Flat,R 25 000,None,Sea Point,2,2,1,None,2023-03-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-112294557,apartment


In [151]:
ht_testet["size"] =ht_testet["size"].str.strip(" m²").str.replace(" ","").astype("float")  # all good
available_from

0       203.0
1         NaN
2       203.0
3        92.0
4         NaN
        ...  
686    1461.0
687      89.0
688     338.0
689     102.0
690      37.0
Name: size, Length: 691, dtype: float64

In [7]:
pp_df.columns

Index(['title', 'priceDescription', 'priceAdditionalDescriptor', 'suburb',
       'number_of_bedrooms', 'number_of_bathrooms', 'number_of_garages',
       'size', 'available_from', 'url'],
      dtype='object')

In [ ]:
'title', 'priceDescription', 'priceAdditionalDescriptor', 'suburb',
       
columns = ["title", 'price', 'additional price info','building type' 'suburb', 'number_of_bedrooms', 'number_of_bathrooms', 'number_of_garages',
       'size', 'available_from', 'url','website','day_scraped']

In [69]:
ht_testet = ht_df.copy()
pp_testet = pp_df.copy()
p24_testet = p24_df.copy()

In [55]:
# I have been doing it seperately, so that would mean I clean each one individually, so it would have to clean before combining

from datetime import date
import numpy as np
def house_type_categories(df):
    """Make categorical column for building type"""
    mask_apt = df['title'].str.contains("Apartment|apartment|flat|Flat")
    mask_house = df['title'].str.contains("house|House")
    mask_commercial = df['title'].str.contains("commercial|Commercial")
    mask_industrial = df['title'].str.contains("industrial|Industrial")
    
    df['building type'] = np.select([mask_apt,mask_house,mask_commercial,mask_industrial],["apartment",'house','commercial property','industrial property'])
    return df
def available_date_cleaner_ht(df):
    
    full_index = list(df.index)
    now_index = list(df.loc[df['available_from'].str.contains("NOW",na=False),"available_from"].index)
    other_index = set(full_index) - set(now_index)
    
    df["available_from"]= df["available_from"].str.strip(" AVAILABLE:")
    df.loc[now_index,"available_from"] = date.today()
    df.loc[other_index,"available_from"] = df.loc[other_index,"available_from"] + " 2023"
    df.loc[other_index,"available_from"] = pd.to_datetime(df.loc[other_index,"available_from"], format = "%d %b %Y",errors='coerce').dt.date
    return df

def num_column_cleaner(df):
    df.loc[df['priceDescription'].str.contains("POA"), 'priceAdditionalDescriptor'] = df['priceAdditionalDescriptor'] + ' POA'
    # check if this POA was solved in the next iteration, i changed the Nones to be """
    df.loc[df['priceDescription'].str.contains("POA"),'priceDescription'] = "0" 

    df['priceDescription'] = df['priceDescription'].str.replace(" ","")
    df['priceDescription'] = df['priceDescription'].str.replace("R","")
    df['priceDescription'] = df['priceDescription'].str.replace("\nperday","")
    df['priceDescription'] = df['priceDescription'].str.replace("\nperm²","").astype("int",errors="ignore")
    df["number_of_bedrooms"]=pd.to_numeric(df['number_of_bedrooms'], errors='coerce').fillna(0)
    df["number_of_bathrooms"]=pd.to_numeric(df['number_of_bathrooms'], errors='coerce').fillna(0)
    df["number_of_garages"]=pd.to_numeric(df['number_of_garages'], errors='coerce').fillna(0)
    
    df["size"] =df["size"].str.strip(" m²").str.replace(" ","")
    df["size"] = pd.to_numeric(df["size"],errors = "coerce").fillna(0)
    return df
ht_clean = house_type_categories(ht_testet)
ht_clean = num_column_cleaner(ht_clean)
ht_clean = available_date_cleaner_ht(ht_clean)
# what about the available dates

In [78]:
# what about the duplicates 


,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url,building type
0,2 Bed Apartment in Sea Point,35000,,Sea Point,2.0,2.0,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3639654,apartment
1,2 Bed Apartment in Sea Point,21500,,Sea Point,2.0,1.0,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/29-kingsgate/301-beach-road/RR3427907,apartment
2,Studio Apartment in Sea Point,15000,DEP R 30K,Sea Point,0.5,1.0,0.0,0.0,2023-04-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3747023,apartment
3,1 Bed Apartment in Sea Point,20000,DEP R 30K,Sea Point,1.0,1.5,1.0,0.0,2023-05-08,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR2718838,apartment
4,3 Bed Apartment in Sea Point,32000,,Sea Point,3.0,3.0,1.0,0.0,2023-04-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3745999,apartment
5,2 Bed Apartment in Sea Point,35000,,Sea Point,2.0,2.0,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3745647,apartment
6,2 Bed Apartment in Sea Point,28000,,Sea Point,2.0,2.5,0.0,0.0,2023-04-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3745186,apartment
7,1 Bed Apartment in Sea Point,15000,,Sea Point,1.0,1.0,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/000-planet-africa/10-apple-kei-road/RR3745172,apartment
8,1 Bed Apartment in Sea Point,13000,,Sea Point,1.0,1.5,0.0,0.0,2023-05-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3744948,apartment
9,3 Bed Apartment in Sea Point,60000,,Sea Point,3.0,2.0,1.0,0.0,2023-04-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3744057,apartment


In [63]:
# time for all the p24 cleaning:
def num_column_cleaner(df)->pd.DataFrame:
    """clean all the other columns"""
    df.loc[df['priceDescription'].str.contains("POA"), 'priceAdditionalDescriptor'] = df['priceAdditionalDescriptor'] + ' POA'
    # check if this POA was solved in the next iteration, i changed the Nones to be """
    df.loc[df['priceDescription'].str.contains("POA"),'priceDescription'] = "0" 

    df['priceDescription'] = df['priceDescription'].str.replace(" ","")
    df['priceDescription'] = df['priceDescription'].str.replace("R","")
    df['priceDescription'] = df['priceDescription'].str.replace("\nperday","")
    df['priceDescription'] = df['priceDescription'].str.replace("\nperm²","").astype("int",errors="ignore")
    df["number_of_bedrooms"]=pd.to_numeric(df['number_of_bedrooms'], errors='coerce').fillna(0)
    df["number_of_bathrooms"]=pd.to_numeric(df['number_of_bathrooms'], errors='coerce').fillna(0)
    df["number_of_garages"]=pd.to_numeric(df['number_of_garages'], errors='coerce').fillna(0)
    
    df["size"] =df["size"].str.strip(" m²").str.replace(" ","")
    df["size"] = pd.to_numeric(df["size"],errors = "coerce").fillna(0)
    return df

def available_date_cleaner_p24(df)->pd.DataFrame:
    full_index = list(df.index)
    now_index = list(df.loc[df['available_from'].str.contains("NOW",na=False),"available_from"].index)
    other_index = set(full_index) - set(now_index)
    
    df["available_from"]= df["available_from"].str.strip("AVAILABLE:")
    df.loc[now_index,"available_from"] = date.today()
    df.loc[other_index,"available_from"] = df.loc[other_index,"available_from"] + " 2023"
    df.loc[other_index,"available_from"] = pd.to_datetime(df.loc[other_index,"available_from"], format = "%d %b %Y",errors='coerce').dt.date
    return df
    
p24_clean = house_type_categories(p24_testet)
p24_clean = num_column_cleaner(p24_clean)
p24_clean = available_date_cleaner_p24(p24_clean)

In [70]:
# Okat time for private property
def num_column_cleaner(df)->pd.DataFrame:
    """clean all the other columns"""
    df.loc[df['priceDescription'].str.contains("POA"), 'priceAdditionalDescriptor'] = df['priceAdditionalDescriptor'] + ' POA'
    # check if this POA was solved in the next iteration, i changed the Nones to be """
    df.loc[df['priceDescription'].str.contains("POA"),'priceDescription'] = "0" 

    df['priceDescription'] = df['priceDescription'].str.replace(" ","")
    df['priceDescription'] = df['priceDescription'].str.replace("R","")
    df['priceDescription'] = df['priceDescription'].str.replace("\nperday","")
    df['priceDescription'] = df['priceDescription'].str.replace("\nperm²","").astype("int",errors="ignore")
    df["number_of_bedrooms"]=pd.to_numeric(df['number_of_bedrooms'], errors='coerce').fillna(0)
    df["number_of_bathrooms"]=pd.to_numeric(df['number_of_bathrooms'], errors='coerce').fillna(0)
    df["number_of_garages"]=pd.to_numeric(df['number_of_garages'], errors='coerce').fillna(0)
    
    df["size"] =df["size"].str.strip(" m²").str.replace(" ","")
    df["size"] = pd.to_numeric(df["size"],errors = "coerce").fillna(0) # Size is alway zero here for private property 
    return df
def available_date_cleaner_pp(df)->pd.DataFrame:
    full_index = list(df.index)
    now_index = list(df.loc[df['available_from'].str.contains("Now",na=False),"available_from"].index)
    other_index = set(full_index) - set(now_index)
    
    df["available_from"]= df["available_from"].str.strip("AVAILABLE FROM: ")
    df.loc[now_index,"available_from"] = date.today()
    df.loc[other_index,"available_from"] = pd.to_datetime(df.loc[other_index,"available_from"], format = "%d %b %Y",errors='coerce').dt.date
    return df 

pp_clean = house_type_categories(pp_testet)
pp_clean = num_column_cleaner(pp_clean)  
pp_clean = available_date_cleaner_pp(pp_clean)


In [71]:
pp_clean.sample(15) # a few missing, but what ya gonna do 

,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url,building type
94,4 Bed Apartment in Green Point,60000,,Green Point,4.0,4.5,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/green-point/RR3732163,apartment
44,4 Bed Apartment in Sea Point,45000,,Sea Point,4.0,3.5,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3724871,apartment
73,1 Bed Apartment in Sea Point,25000,,Sea Point,1.0,1.0,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3642169,apartment
97,2 Bed Apartment in Green Point,32000,,Green Point,2.0,2.0,2.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/green-point/RR3730676,apartment
66,3 Bed Apartment in Sea Point,75000,,Sea Point,3.0,3.5,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3698325,apartment
21,2 Bed Apartment in Sea Point,25000,,Sea Point,2.0,1.0,0.0,0.0,2023-04-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3740796,apartment
74,1 Bed Flat in Sea Point,7700,DEP R 15K,Sea Point,1.0,1.0,0.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR2459315,apartment
10,2 Bed Apartment in Sea Point,42000,DEP R 84K,Sea Point,2.0,2.0,4.0,0.0,2023-03-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/21-lisdale/259-beach-road/RR3603135,apartment
39,4 Bed Apartment in Sea Point,60000,DEP R 120K,Sea Point,4.0,4.5,0.0,0.0,2023-05-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3729804,apartment
2,Studio Apartment in Sea Point,15000,DEP R 30K,Sea Point,0.5,1.0,0.0,0.0,2023-04-01,https://www.privateproperty.co.za/to-rent/western-cape/cape-town/atlantic-seaboard/sea-point/RR3747023,apartment


In [346]:
# my goddam Urls are not correct for Hometraders. 
ht_df.loc[383]

title                                                                                       1 Bedroom Apartment / Flat
priceDescription                                                                                              R 12 750
priceAdditionalDescriptor                                                                                         None
suburb                                                                                                     Green Point
number_of_bedrooms                                                                                                   1
number_of_bathrooms                                                                                                  1
number_of_garages                                                                                                    1
size                                                                                                             80 m²
available_from                                  

In [343]:
ht_clean[ht_clean.priceDescription < 5000].sample(15)

,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url,building type
252,Commercial Property,170,per m²,Sea Point,NaN,NaN,NaN,514.0,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-111035825,commercial property
267,2 Bedroom Apartment / Flat,2500,per day,Sea Point,2.0,2.0,2.0,NaN,None,https://www.sahometraders.co.za/commercial-property-to-rent-in-sea-point-111035825,apartment
497,Commercial Property,200,per m²,Green Point,NaN,NaN,NaN,285.0,None,https://www.sahometraders.co.za/commercial-property-to-rent-in-green-point-112349623,commercial property
672,Commercial Property,198,per m²,Gardens,NaN,NaN,NaN,65.0,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110633270,commercial property
684,Commercial Property,150,per m²,Gardens,NaN,NaN,NaN,190.0,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110633270,commercial property
664,Commercial Property,198,per m²,Gardens,NaN,NaN,NaN,38.0,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110456444,commercial property
483,5 Bedroom House,0,per day POA,Green Point,5.0,5.0,3.0,NaN,None,https://www.sahometraders.co.za/commercial-property-to-rent-in-green-point-111641745,house
681,Commercial Property,130,per m²,Gardens,NaN,NaN,NaN,827.0,None,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110633270,commercial property
667,Commercial Property,180,per m²,Gardens,NaN,NaN,NaN,180.0,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110456444,commercial property
149,2 Bedroom House,0,NaN,Sea Point,2.0,2.0,NaN,183.0,AVAILABLE: 02 MAR,https://www.sahometraders.co.za/3-bedroom-apartment-flat-to-rent-in-sea-point-112178264,house


,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url,building type
51,Commercial Property,R 170\n per m²,per m²,Sea Point,None,None,None,None,AVAILABLE NOW,https://www.sahometraders.co.za/2-bedroom-apartment-flat-to-rent-in-sea-point-112328991,commercial property
116,Apartment / Flat,R 700,None,Sea Point,None,1,None,1 600 m²,AVAILABLE: 01 MAR,https://www.sahometraders.co.za/3-bedroom-apartment-flat-to-rent-in-sea-point-112234972,apartment
134,Commercial Property,R 175\n per m²,per m²,Sea Point,None,None,None,350 m²,AVAILABLE NOW,https://www.sahometraders.co.za/apartment-flat-to-rent-in-sea-point-111878609,commercial property
149,2 Bedroom House,POA,None,Sea Point,2,2,None,183 m²,AVAILABLE: 02 MAR,https://www.sahometraders.co.za/3-bedroom-apartment-flat-to-rent-in-sea-point-112178264,house
150,3 Bedroom Apartment / Flat,POA\n per day,per day,Sea Point,3,3,1,149 m²,None,https://www.sahometraders.co.za/3-bedroom-apartment-flat-to-rent-in-sea-point-112178264,apartment
...,...,...,...,...,...,...,...,...,...,...,...
671,Commercial Property,R 58\n per m²,per m²,Gardens,None,None,None,750 m²,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110633270,commercial property
672,Commercial Property,R 198\n per m²,per m²,Gardens,None,None,None,65 m²,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110633270,commercial property
679,Commercial Property,R 160\n per m²,per m²,Gardens,None,None,None,225 m²,AVAILABLE NOW,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110633270,commercial property
681,Commercial Property,R 130\n per m²,per m²,Gardens,None,None,None,827 m²,None,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-110633270,commercial property


In [96]:
import pandas as pd
notraw_df = pd.read_parquet("D:\Real estate data engineering project\saved_files\Full_clean_Scrape20230301")
notraw_df.sample(1)


,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url,building type
605,Commercial Property,15000,,Gardens,0.0,0.0,0.0,34.0,2023-03-01,https://www.sahometraders.co.za/commercial-property-to-rent-in-gardens-112223773,commercial property


In [97]:


import pandas as pd
notraw_df.sample(1).stack()


112  title                                                                                     3 Bedroom Apartment / Flat
     priceDescription                                                                                               45000
     priceAdditionalDescriptor                                                                                           
     suburb                                                                                                     Sea Point
     number_of_bedrooms                                                                                               3.0
     number_of_bathrooms                                                                                              2.0
     number_of_garages                                                                                                2.0
     size                                                                                                           170.0
     available_from     